# Amazom Mobiles Data Cleaning


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns

#Libraries additionally added
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import ast

#Regular Expressions
import re

# Study data files
iphone_data_path = "../Scraped Data/scraped_iphone_data.csv"
samsung_data_path = "../Scraped Data/scraped_samsung_data.csv"


# Read the Iphone & Samsung data 
iphone_df = pd.read_csv(iphone_data_path)
samsung_df = pd.read_csv(samsung_data_path)




In [2]:
#Display the Iphone DataFrame
iphone_df.head()

,title,URL,product_title,price,star_ratings,number_of_global_ratings,customer_stars_percentages,Features_1,Features_2,Features_3
0,"Apple iPhone 12, 64GB, Blue - Fully Unlocked (...",https://www.amazon.ca/Apple-iPhone-12-64GB-Blu...,"Apple iPhone 12, 64GB, Blue - Fully Unlocked (...",$435.00,4.1 out of 5 stars,"12,493 global ratings","{'5 star': '62%', '4 star': '15%', '3 star': '...",Apple iPhone 12,64GB,Blue - Fully Unlocked (Renewed)
1,"Apple iPhone 12 Pro, 128GB, Graphite - Fully U...",https://www.amazon.ca/Apple-iPhone-Pro-128GB-G...,"Apple iPhone 12 Pro, 128GB, Graphite - Fully U...",$689.86,4.1 out of 5 stars,"4,711 global ratings","{'5 star': '59%', '4 star': '17%', '3 star': '...",Apple iPhone 12 Pro,128GB,Graphite - Fully Unlocked (Renewed)
2,"Apple iPhone 13 Pro, 128GB, Gold (Renewed)",https://www.amazon.ca/Apple-iPhone-13-Pro-Rene...,"Apple iPhone 13 Pro, 128GB, Gold (Renewed)",$799.99,4.3 out of 5 stars,161 global ratings,"{'5 star': '70%', '4 star': '13%', '3 star': '...",Apple iPhone 13 Pro,128GB,Gold (Renewed)
3,"Apple iPhone 13 Pro Max, 128GB, Sierra Blue - ...",https://www.amazon.ca/Apple-iPhone-13-Pro-Max/...,"Apple iPhone 13 Pro Max, 128GB, Sierra Blue - ...",$899.99,4.3 out of 5 stars,420 global ratings,"{'5 star': '71%', '4 star': '12%', '3 star': '...",Apple iPhone 13 Pro Max,128GB,Sierra Blue - Unlocked (Renewed)
4,"Apple iPhone 14, 128GB, Midnight - Unlocked (R...",https://www.amazon.ca/Apple-iPhone-14-128GB-Mi...,"Apple iPhone 14, 128GB, Midnight - Unlocked (R...",$899.98,4.4 out of 5 stars,852 global ratings,"{'5 star': '75%', '4 star': '10%', '3 star': '...",Apple iPhone 14,128GB,Midnight - Unlocked (Renewed)


# Clean the Iphone DataFrame

# Format, Drop, Update Columns 

In [3]:
#Check the data types
iphone_df.dtypes

title                         object
URL                           object
product_title                 object
price                         object
star_ratings                  object
number_of_global_ratings      object
customer_stars_percentages    object
Features_1                    object
Features_2                    object
Features_3                    object
dtype: object

In [4]:
# Rename the 'Features_1', 'Features_2', and 'Features_3' columns
iphone_df = iphone_df.rename(columns={
    'Features_1': 'brand_model',
    'Features_2': 'storage_capacity',
    'Features_3': 'color'
})



In [5]:

# Drop the 'title', 'product_title', and 'url' columns
iphone_df = iphone_df.drop(['title', 'product_title', 'URL'], axis=1)

# Create  'storage_capacity' column by extracting the storage capacity
iphone_df['storage_capacity'] = iphone_df['storage_capacity'].str.replace('GB', '').astype(int)

# Extract color information from the 'color' column using a regular expression
iphone_df['color'] = iphone_df['color'].str.extract(r'([a-zA-Z\s]+)').squeeze()

# Remove dollar signs and commas from the 'price' column and convert to float
iphone_df['price'] = iphone_df['price'].str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype(float)

# Extract the numerical part of the 'star_ratings' column and convert to float
iphone_df['star_ratings'] = iphone_df['star_ratings'].str.split(' ', expand=True)[0].astype(float)

# Extract only the numeric values from the 'number_of_global_ratings' column
iphone_df['number_of_global_ratings'] = iphone_df['number_of_global_ratings'].str.replace(',', '')
iphone_df['number_of_global_ratings'] = iphone_df['number_of_global_ratings'].str.extract('(\d+)').astype(int)


In [6]:

# Convert string representation of dictionaries in 'customer_stars_percentages' to actual dictionaries
iphone_df['customer_stars_percentages'] = iphone_df['customer_stars_percentages'].apply(ast.literal_eval)

# Convert the 'customer_stars_percentages' column into a DataFrame with separate columns for each star rating
stars_df = pd.json_normalize(iphone_df['customer_stars_percentages'])

# Concatenate the new DataFrame with the original DataFrame
iphone_df = pd.concat([iphone_df, stars_df], axis=1)

# Drop the original 'customer_stars_percentages' columns
iphone_df = iphone_df.drop('customer_stars_percentages', axis=1)


In [7]:
# Remove percentages from '5 star' to '1 star' columns
columns_to_clean = ['5 star', '4 star', '3 star', '2 star', '1 star']

for column in columns_to_clean:
    iphone_df[column] = iphone_df[column].str.replace('%', '').astype(int)

In [8]:

# List of manual model year values corresponding to each row
iphone_model_years = [2020, 2020,  2021, 2021, 2022, 2022]

# For iPhone dataframe
iphone_df['model_year'] = iphone_model_years


In [9]:
# Reorder the columns names as desired
desired_order = ['brand_model', 
                 'storage_capacity', 
                 'color',
                 'model_year',
                 'price',
                 'number_of_global_ratings', 
                 'star_ratings', 
                 '5 star', '4 star', '3 star', '2 star', '1 star']

iphone_df = iphone_df[desired_order]




In [10]:
# Confirm the datatypes updated
iphone_df.dtypes

brand_model                  object
storage_capacity              int64
color                        object
model_year                    int64
price                       float64
number_of_global_ratings      int64
star_ratings                float64
5 star                        int64
4 star                        int64
3 star                        int64
2 star                        int64
1 star                        int64
dtype: object

In [11]:
# Display the resulting DataFrame
iphone_df



,brand_model,storage_capacity,color,model_year,price,number_of_global_ratings,star_ratings,5 star,4 star,3 star,2 star,1 star
0,Apple iPhone 12,64,Blue,2020,435.00,12493,4.1,62,15,6,4,12
1,Apple iPhone 12 Pro,128,Graphite,2020,689.86,4711,4.1,59,17,6,4,13
2,Apple iPhone 13 Pro,128,Gold,2021,799.99,161,4.3,70,13,4,4,8
3,Apple iPhone 13 Pro Max,128,Sierra Blue,2021,899.99,420,4.3,71,12,3,3,10
4,Apple iPhone 14,128,Midnight,2022,899.98,852,4.4,75,10,4,2,9
5,Apple iPhone 14 Pro Max,128,Deep Purple,2022,1849.00,669,4.3,73,8,4,2,13


# Samsung DataFrame

In [12]:
#Display the Samsung DataFrame
samsung_df.head()

,title,URL,product_title,price,star_ratings,number_of_global_ratings,customer_stars_percentages
0,"Samsung Galaxy S20 FE (5G) 128GB 6.5"" Display ...",https://www.amazon.ca/Samsung-Display-Factory-...,"Samsung Galaxy S20 FE (5G) 128GB 6.5"" Display ...",$269.00,3.9 out of 5 stars,689 global ratings,"{'5 star': '58%', '4 star': '14%', '3 star': '..."
1,"Samsung Galaxy S21 5G 128GB 6.2"" Display, Snap...",https://www.amazon.ca/Samsung-Galaxy-S21-5G-Sn...,"Samsung Galaxy S21 5G 128GB 6.2"" Display, Snap...",$294.99,3.6 out of 5 stars,173 global ratings,"{'5 star': '48%', '4 star': '13%', '3 star': '..."
2,Samsung Galaxy S22 (5G) 128GB Unlocked - Phant...,https://www.amazon.ca/Samsung-Galaxy-S22-128GB...,Samsung Galaxy S22 (5G) 128GB Unlocked - Phant...,$476.91,4.1 out of 5 stars,105 global ratings,"{'5 star': '63%', '4 star': '11%', '3 star': '..."
3,Samsung Galaxy S22 Ultra (5G) 128GB Unlocked -...,https://www.amazon.ca/Samsung-Galaxy-Ultra-128...,Samsung Galaxy S22 Ultra (5G) 128GB Unlocked -...,$839.99,4.0 out of 5 stars,28 global ratings,"{'5 star': '56%', '4 star': '19%', '3 star': '..."
4,"Samsung Galaxy S23 5G Black 128GB - 6.1"" 120 H...",https://www.amazon.ca/Samsung-Galaxy-S23-Black...,"Samsung Galaxy S23 5G Black 128GB - 6.1"" 120 H...",$877.96,4.5 out of 5 stars,546 global ratings,"{'5 star': '78%', '4 star': '11%', '3 star': '..."


# Clean the Samsung  DataFrame

# Format, Drop, Update Columns 

In [13]:
#Check the data types
samsung_df.dtypes

title                         object
URL                           object
product_title                 object
price                         object
star_ratings                  object
number_of_global_ratings      object
customer_stars_percentages    object
dtype: object

In [14]:

def extract_phone_model(url):
    match = re.search(r'Samsung[\w\s\-\(\)]+(?=\d+GB)', url)
    return match.group(0).replace('-', ' ').strip() if match else 'Unknown'

def extract_size(url):
    match = re.search(r'\d+GB', url)
    return match.group(0) if match else 'Unknown'


def extract_color(url):
    colors = ['Black', 'Navy', 'White', 'Red', 'Blue', 'Green', 'Yellow', 'Purple', 'Pink', 'Grey', 'Silver', 'Gold', 'Bronze']
    for color in colors:
        if color.lower() in url.lower():
            return color
    return 'Unknown'

In [15]:
samsung_df['titlenew']=samsung_df['title'].apply(extract_phone_model).str.split().str[0:4].apply(lambda x: ' '.join(x))

In [16]:
samsung_df['size']=samsung_df['title'].apply(extract_size).str[0:3].astype('int64')

In [17]:
samsung_df['color']=samsung_df['title'].apply(extract_color)


In [18]:

# Drop the 'title', 'product_title', and 'url' columns
samsung_df = samsung_df.drop(['title', 'product_title', 'URL'], axis=1)


# Remove dollar signs and commas from the 'price' column and convert to float
samsung_df['price'] = samsung_df['price'].str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype(float)

# Extract the numerical part of the 'star_ratings' column and convert to float
samsung_df['star_ratings'] = samsung_df['star_ratings'].str.split(' ', expand=True)[0].astype(float)

# Extract only the numeric values from the 'number_of_global_ratings' column
samsung_df['number_of_global_ratings'] = samsung_df['number_of_global_ratings'].str.replace(',', '')
samsung_df['number_of_global_ratings'] = samsung_df['number_of_global_ratings'].str.extract('(\d+)').astype(int)


In [19]:

# Convert string representation of dictionaries in 'customer_stars_percentages' to actual dictionaries
samsung_df['customer_stars_percentages'] = samsung_df['customer_stars_percentages'].apply(ast.literal_eval)

# Convert the 'customer_stars_percentages' column into a DataFrame with separate columns for each star rating
stars_df = pd.json_normalize(samsung_df['customer_stars_percentages'])

# Concatenate the new DataFrame with the original DataFrame
samsung_df = pd.concat([samsung_df, stars_df], axis=1)

# Drop the original 'customer_stars_percentages' column
samsung_df = samsung_df.drop('customer_stars_percentages', axis=1)



In [20]:
# Remove percentages from '5 star' to '1 star' columns
columns_to_clean = ['5 star', '4 star', '3 star', '2 star', '1 star']

for column in columns_to_clean:
    samsung_df[column] = samsung_df[column].str.replace('%', '').astype(int)

In [21]:

# Rename the 'titlenew' and 'size' columns
samsung_df= samsung_df.rename(columns={
    'titlenew': 'brand_model',
    'size': 'storage_capacity'
})



In [22]:

# List of manual model year values corresponding to each row
samsung_model_years = [2020, 2021, 2022, 2022, 2023, 2023]

# For Samsung dataframe
samsung_df['model_year'] = samsung_model_years

In [23]:

# Reorder the columns names as desired
desired_order = ['brand_model', 
                 'storage_capacity', 
                 'color',
                 'model_year',
                 'price', 
                 'number_of_global_ratings', 
                 'star_ratings', 
                 '5 star', '4 star', '3 star', '2 star', '1 star']

samsung_df = samsung_df[desired_order]



In [24]:
# Confirm the updated data types 
samsung_df.dtypes

brand_model                  object
storage_capacity              int64
color                        object
model_year                    int64
price                       float64
number_of_global_ratings      int64
star_ratings                float64
5 star                        int64
4 star                        int64
3 star                        int64
2 star                        int64
1 star                        int64
dtype: object

In [25]:
# Display the resulting DataFrame
samsung_df

,brand_model,storage_capacity,color,model_year,price,number_of_global_ratings,star_ratings,5 star,4 star,3 star,2 star,1 star
0,Samsung Galaxy S20 FE,128,Navy,2020,269.00,689,3.9,58,14,5,6,17
1,Samsung Galaxy S21 5G,128,Grey,2021,294.99,173,3.6,48,13,9,11,19
2,Samsung Galaxy S22 (5G),128,Black,2022,476.91,105,4.1,63,11,11,6,10
3,Samsung Galaxy S22 Ultra,128,Black,2022,839.99,28,4.0,56,19,7,0,18
4,Samsung Galaxy S23 5G,128,Black,2023,877.96,546,4.5,78,11,3,2,7
5,Samsung Galaxy S23 Ultra,256,Black,2023,1399.00,47,4.3,72,8,5,3,11


# Save iphone_df and samsung_df to JSON and CSV 

In [26]:
samsung_df.to_json('../Cleaned Data/samsung_cleaned_data.json', orient='records', lines=True)
samsung_df.to_csv('../Cleaned Data/samsung_cleaned_data.csv', index=False)

In [27]:
iphone_df.to_json('../Cleaned Data/iphone_cleaned_data.json', orient='records', lines=True)
iphone_df.to_csv('../Cleaned Data/iphone_cleaned_data.csv', index=False)